# Задание 1.

Обучите нейронную сеть решать шифр Цезаря.

In [ ]:
import pandas as pd
import time
import torch
from sklearn.metrics import accuracy_score

In [ ]:
def caesar(text, key = 2, alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'):
    """
    Функция реализует шифр Цезаря.
    если алфавит не передан, то используется
    английский алфавит
    """
    res = ''
    for i in text:
      if str(i).isalpha():
        if i.upper() in alphabet:
            buk = i.upper()
            new_buk = alphabet[((alphabet.index(buk) + key) % len(alphabet))]
            res += new_buk.lower()
      else:
        res += str(i)
    return(res)

Для обучения нейросети возьмем датасет с выражениями героев сериала “Симпсоны”

In [ ]:
df = pd.read_csv('data.csv')

In [ ]:
df.head()

,Unnamed: 0,id,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count
0,0,10368,35,29,"Lisa Simpson: Maggie, look. What's that?",235000,True,9,5.0,Lisa Simpson,Simpson Home,"Maggie, look. What's that?",maggie look whats that,4.0
1,1,10369,35,30,Lisa Simpson: Lee-mur. Lee-mur.,237000,True,9,5.0,Lisa Simpson,Simpson Home,Lee-mur. Lee-mur.,lee-mur lee-mur,2.0
2,2,10370,35,31,Lisa Simpson: Zee-boo. Zee-boo.,239000,True,9,5.0,Lisa Simpson,Simpson Home,Zee-boo. Zee-boo.,zee-boo zee-boo,2.0
3,3,10372,35,33,Lisa Simpson: I'm trying to teach Maggie that ...,245000,True,9,5.0,Lisa Simpson,Simpson Home,I'm trying to teach Maggie that nature doesn't...,im trying to teach maggie that nature doesnt e...,24.0
4,4,10374,35,35,"Lisa Simpson: It's like an ox, only it has a h...",254000,True,9,5.0,Lisa Simpson,Simpson Home,"It's like an ox, only it has a hump and a dewl...",its like an ox only it has a hump and a dewlap...,18.0


In [ ]:
phrases = df['normalized_text'].tolist()

In [ ]:
text = [ph for ph in phrases if type(ph) is str]

In [ ]:
train = pd.DataFrame({'text': text, 'caesar_text': [caesar(i) for i in text]})

In [ ]:
train.head()

,text,caesar_text
0,maggie look whats that,ociikg nqqm yjcvu vjcv
1,lee-mur lee-mur,ngg-owt ngg-owt
2,zee-boo zee-boo,bgg-dqq bgg-dqq
3,im trying to teach maggie that nature doesnt e...,ko vtakpi vq vgcej ociikg vjcv pcvwtg fqgupv g...
4,its like an ox only it has a hump and a dewlap...,kvu nkmg cp qz qpna kv jcu c jwor cpf c fgyncr...


In [ ]:
text_x = [[j for j in i] for i in train['caesar_text']]

In [ ]:
text_y = [[j for j in i] for i in train['text']]

In [ ]:
CHARS = set('abcdefghijklmnopqrstuvwxyz ')  # все символы, которые мы хотим использовать для кодировки = наш словарь
INDEX_TO_CHAR = ['none'] + [w for w in CHARS]  # все неизвестные символы будут получать тег none
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}  # словарь токен-индекс

In [ ]:
MAX_LEN = len(max(text,key = len)) # максимальная длина текста
MAX_LEN

396

In [ ]:
X = torch.zeros((len(text_x), MAX_LEN), dtype=int)  # создаём пустой вектор для текста, чтобы класть в него индексы токенов
for i in range(len(text_x)):  # для каждого предложения
    for j, w in enumerate(text_x[i]):  # для каждого токена
        if j >= MAX_LEN:
            break
        X[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

In [ ]:
y = torch.zeros((len(text_y), MAX_LEN), dtype=int)
for i in range(len(text_y)):
    for j, w in enumerate(text_y[i]):
        if j >= MAX_LEN:
            break
        y[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

In [ ]:
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.embedding = torch.nn.Embedding(28, 30)
        self.rnn = torch.nn.RNN(30, 128)
        self.out = torch.nn.Linear(128, 28)

    def forward(self, sentences, state=None):
        x = self.embedding(sentences)
        x, s = self.rnn(x) # берём выход с последнего слоя для всех токенов, а не скрытое состояние
        return self.out(x)

In [ ]:
model = Network()

In [ ]:
criterion = torch.nn.CrossEntropyLoss()  # типичный лосс многоклассовой классификации
optimizer = torch.optim.SGD(model.parameters(), lr=.05)

In [ ]:
for ep in range(20):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    for i in range(int(len(X) / 100)):
        # берём батч в 100 элементов
        X_batch = X[i * 100:(i + 1) * 100]
        Y_batch = y[i * 100:(i + 1) * 100].flatten()

        model.train()

        optimizer.zero_grad()
        answers = model.forward(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(answers, Y_batch)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))

Epoch 0. Time: 15.166, Train loss: 0.450
Epoch 1. Time: 15.276, Train loss: 0.286
Epoch 2. Time: 15.355, Train loss: 0.226
Epoch 3. Time: 15.361, Train loss: 0.183
Epoch 4. Time: 15.074, Train loss: 0.150
Epoch 5. Time: 15.276, Train loss: 0.124
Epoch 6. Time: 14.934, Train loss: 0.105
Epoch 7. Time: 15.000, Train loss: 0.090
Epoch 8. Time: 15.284, Train loss: 0.078
Epoch 9. Time: 15.002, Train loss: 0.068
Epoch 10. Time: 14.970, Train loss: 0.060
Epoch 11. Time: 15.212, Train loss: 0.053
Epoch 12. Time: 15.213, Train loss: 0.047
Epoch 13. Time: 16.071, Train loss: 0.043
Epoch 14. Time: 15.041, Train loss: 0.038
Epoch 15. Time: 15.589, Train loss: 0.035
Epoch 16. Time: 15.051, Train loss: 0.032
Epoch 17. Time: 14.947, Train loss: 0.029
Epoch 18. Time: 15.257, Train loss: 0.027
Epoch 19. Time: 15.034, Train loss: 0.025


In [ ]:
def generate_sentence(word):
    sentence = list(word)
    sentence = [CHAR_TO_INDEX.get(s, 0) for s in sentence]
    answers = model.forward(torch.tensor(sentence))
    probas, indices = answers.topk(1)
    return ''.join([INDEX_TO_CHAR[ind.item()] for ind in indices.flatten()])

In [ ]:
generate_sentence('ociikg nqqm yjcvu vjcv')

'maggie look whats that'

In [ ]:
train['text_predict'] = train['caesar_text'].apply(generate_sentence)

In [ ]:
train.head(50)

,text,caesar_text,text_predict
0,maggie look whats that,ociikg nqqm yjcvu vjcv,maggie look whats that
1,lee-mur lee-mur,ngg-owt ngg-owt,leenonemur leenonemur
2,zee-boo zee-boo,bgg-dqq bgg-dqq,noneeenoneboo oeenoneboo
3,im trying to teach maggie that nature doesnt e...,ko vtakpi vq vgcej ociikg vjcv pcvwtg fqgupv g...,im trying to teach maggie that nature doesnt e...
4,its like an ox only it has a hump and a dewlap...,kvu nkmg cp qz qpna kv jcu c jwor cpf c fgyncr...,its like an od only it has a hump and a dewlap...
5,you know his blood type how romantic,aqw mpqy jku dnqqf varg jqy tqocpvke,you know his blood type how romantic
6,oh yeah whats my shoe size,qj agcj yjcvu oa ujqg ukbg,oh yeah whats my shoe sioe
7,ring,tkpi,ring
8,yes dad,agu fcf,yes dad
9,ooh look maggie what is that do-dec-ah-edron d...,qqj nqqm ociikg yjcv ku vjcv fq-fge-cj-gftqp f...,ooh look maggie what is that dononedecnoneahno...


# Задание 2

Построить RNN-ячейку на основе полносвязных слоев.
Применить построенную ячейку для генерации текста с выражениями героев сериала “Симпсоны”

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
class RNN(torch.nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        self.embedding = torch.nn.Embedding(28, 30)
        self.rnn = torch.nn.RNN(30, 256)
        self.out = torch.nn.Linear(256, 28)

    def forward(self, sentences, state=None):
        x = self.embedding(sentences)
        x, s = self.rnn(x) # берём выход с последнего слоя для всех токенов, а не скрытое состояние
        return self.out(x)

In [ ]:
model_RNN = RNN().to(device)

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_RNN.parameters(), lr=.05)

In [ ]:
for ep in range(200):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    for i in range(int(len(X) / 100)):
        # берём батч в 100 элементов
        batch = X[i * 100:(i + 1) * 100]
        X_batch = batch[:, :-1].to(device)
        Y_batch = batch[:, 1:].flatten().to(device)

        optimizer.zero_grad()
        answers = model_RNN.forward(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(answers, Y_batch).to(device)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))

Epoch 0. Time: 1.492, Train loss: 0.321
Epoch 1. Time: 1.394, Train loss: 0.319
Epoch 2. Time: 1.428, Train loss: 0.317
Epoch 3. Time: 1.400, Train loss: 0.316
Epoch 4. Time: 1.399, Train loss: 0.314
Epoch 5. Time: 1.400, Train loss: 0.313
Epoch 6. Time: 1.408, Train loss: 0.312
Epoch 7. Time: 1.437, Train loss: 0.311
Epoch 8. Time: 1.421, Train loss: 0.310
Epoch 9. Time: 1.421, Train loss: 0.309
Epoch 10. Time: 1.417, Train loss: 0.309
Epoch 11. Time: 1.425, Train loss: 0.308
Epoch 12. Time: 1.421, Train loss: 0.307
Epoch 13. Time: 1.418, Train loss: 0.307
Epoch 14. Time: 1.424, Train loss: 0.306
Epoch 15. Time: 1.450, Train loss: 0.306
Epoch 16. Time: 1.457, Train loss: 0.305
Epoch 17. Time: 1.437, Train loss: 0.305
Epoch 18. Time: 1.439, Train loss: 0.304
Epoch 19. Time: 1.444, Train loss: 0.304
Epoch 20. Time: 1.443, Train loss: 0.303
Epoch 21. Time: 1.445, Train loss: 0.303
Epoch 22. Time: 1.452, Train loss: 0.303
Epoch 23. Time: 1.459, Train loss: 0.302
Epoch 24. Time: 1.466, Tra

In [ ]:
model = model_RNN.to('cpu')

In [ ]:
generate_sentence('i dont think')

' vgg gvg  np'